# I FUNCTIONAL APIs


**IO API**, the functional way 

In [10]:
trait IO[P[_]]{
  def read(): P[String]
  def write(msg: String): P[Unit]
    
  // Imperative combinators
  def doAndThen[A, B](p: P[A])(f: A => P[B]): P[B]
  def returns[A](a: A): P[A]
}

defined trait IO

**Asynchronous API**, modularised

In [23]:
import scala.concurrent.Future

type AsyncIO = IO[Future]

import scala.concurrent.Future


defined type AsyncIO

**State-transformations**, modularised


In [12]:
case class IOState(reads: List[String], writes: List[String])

type IOTrans[T] = IOState => (IOState, T)

defined class IOState
defined type IOTrans

**Synchronous API**, modularised

In [13]:
type Id[T] = T
type SyncIO = IO[Id]

defined type Id
defined type SyncIO

# II API INSTANTIATIONS

**Asynchronous** instance

In [24]:
import scala.concurrent.ExecutionContext
import ExecutionContext.Implicits.global

object AsyncIO extends IO[Future]{
  def read(): Future[String] = ??? // Whatever
  def write(msg: String): Future[Unit] = ??? // Whatever
  
  // Imperative combinators
  def doAndThen[A, B](p: Future[A])(f: A => Future[B]): Future[B] = 
    p flatMap f
    
  def returns[A](a: A): Future[A] = 
    Future.successful(a)
}

import scala.concurrent.ExecutionContext

import ExecutionContext.Implicits.global


defined object AsyncIO

**Synchronous** interpretation of IO Programs

In [25]:
import scala.io.StdIn.readLine

object ConsoleIO extends IO[Id]{
  def read(): String = readLine()
  def write(msg: String): Unit = println(msg)
    
  // Imperative combinators
  def doAndThen[A, B](p: A)(f: A => B): B = 
    f(p)
  def returns[A](a: A): A = 
    a
}

import scala.io.StdIn.readLine


defined object ConsoleIO

**State-based** interpretation

In [16]:
object StateIO extends IO[IOTrans]{
  def read(): IOState => (IOState, String) = ???
  def write(msg: String): IOState => (IOState, Unit) = ???
    
  // Imperative combinators
  def doAndThen[A,B](p: IOTrans[A])(f: A => IOTrans[B]): IOTrans[B] =
    iostate1 => p(iostate1) match {
      case (iostate2, a) => f(a)(iostate2)
    }

  def returns[A](a: A): IOTrans[A] =
    iostate => (iostate, a)
}

defined object StateIO

# III LOGIC

In [17]:
def echo[P[_]]()(io: IO[P]): P[String] = 
  io.doAndThen(io.read()){ msg: String => 
    io.doAndThen(io.write(msg)){ _ : Unit => 
      io.returns(msg)
    }
  }

defined function echo

# IV COMPOSITION

**Synchronous** echo, modularised

In [18]:
def consoleEcho(): String =
  echo()(ConsoleIO)

defined function consoleEcho

**State-based** echo, modularised

In [19]:
def stateEcho(): IOState => (IOState, String) = 
  ???

defined function stateEcho

**Asynchronous** echo, modularised

In [21]:
def asynEcho(): Future[String] = 
  ???

defined function asynEcho